In [1]:
import requests # Apache License 2.0
from requests.auth import HTTPBasicAuth

import uuid     # in python
import base64   # in python
import yaml     # MIT

from app.utils import make_create_secure_asset_body, create_generic_Access_PolicyDefinitionRequest_body, create_generic_Usage_PolicyDefinitionRequest_body, create_generic_ContractDefinitionRequest_body, print_edc_assets

In [2]:
# This script creates an asset for a submodel in the Provider-EDC
# + access and usage policies
# + contract binding all three parts together

In [4]:
# --- variables ---
with open('provider_cfg.yaml', 'r') as file:
    consumer_cfg = yaml.safe_load(file)

# - control plane -
url_edc_provider_control_plane_base = consumer_cfg['provider-edc-control-plane']['endpoint']
header_control_plane = consumer_cfg['provider-edc-control-plane']['header'] # this contains secrets, so please use -at least- a secretsmanager instead

# construct urls based on standard:
url_edc_provider_asset_management    = url_edc_provider_control_plane_base + "/management/v3/assets"
url_edc_provider_policy_management   = url_edc_provider_control_plane_base + "/management/v2/policydefinitions"
url_edc_provider_contract_management = url_edc_provider_control_plane_base + "/management/v2/contractdefinitions"


# - "identities" -
edc_consumer_bpn = consumer_cfg['trusted-consumers-bpn']['machine_builder_A']


# - submodel repository -
submodel_repo_url = consumer_cfg['submodel-repository']['endpoint'] 
proxy_auth_header = consumer_cfg['submodel-repository']['authtoken'] 

In [ ]:
# look up the existing assets:
res_get_assets = requests.post(url_edc_provider_asset_management + "/request", headers=header_control_plane)
print_edc_assets(res_get_assets)

In [ ]:
# - create EDC-Asset -
# the asset is here a submodel
target_url = submodel_repo_url + "/submodels/" + "MWQzOGE4ZDgtYzY3Ni00ZDliLWIxNTEtMjc3OTUzZDNiM2E1"

edc_asset_id   = "1d38a8d8-c676-4d9b-b151-277953d3b3a5"
edc_asset_type = "Submodel" # according to https://w3id.org/catenax/taxonomy#Submodel


create_asset_body = make_create_secure_asset_body(edc_asset_id, edc_asset_type, target_url, proxy_auth_header)
# register asset:
res_create_edc_asset = requests.post(url_edc_provider_asset_management, headers=header_control_plane, json=create_asset_body)
res_create_edc_asset

In [ ]:
# - cheack if the asset exists already -
# look up the existing assets (not sure why a post again):
res_get_assets = requests.post(url_edc_provider_asset_management + "/request", headers=header_control_plane)

if res_get_assets.status_code == 200:
    asset_ids = [retrived_asset["@id"] for retrived_asset in res_get_assets.json()]
    if  edc_asset_id in asset_ids:
        print('Asset already exits ... resolving conflict via update')
    else:
        print("Asset doesn't exist, conflict reason unknown.")

else:
    print('Request Error: ' + str(res_get_assets.status_code))

In [ ]:
# create:
res_update_edc_asset = requests.put(url_edc_provider_asset_management, headers=header_control_plane, json=create_asset_body)
res_update_edc_asset

In [ ]:
# look up the existing assets (not sure why a post again):
res_get_assets = requests.post(url_edc_provider_asset_management + "/request", headers=header_control_plane)
print_edc_assets(res_get_assets)

In [8]:
# # remove assets:
# asset_deletion_body = {
#     "@context": "https://w3id.org/edc/v0.0.1/ns/",    
#     "@id": "test_AAS_registration_C_3",
# }
# res_delete_edc_asset = requests.delete(url_edc_provider_asset_management, headers=header_control_plane, json=create_asset_body)
# res_delete_edc_asset

In [9]:
# ------------------------------------------------------------------------------------------------------

### Assing a Policy to the created Asset

In [ ]:
# create access policy:
create_access_policy_body = create_generic_Access_PolicyDefinitionRequest_body("access-policy-generic", edc_consumer_bpn, policy_action="use")

res_create_asset_access_plicy = requests.post(url_edc_provider_policy_management, headers=header_control_plane, json=create_access_policy_body)
res_create_asset_access_plicy # post against: "{{EDCTX-10-1-URL}}/management/v2/policydefinitions"

In [ ]:
# create usage policy:
create_usage_policy_body = create_generic_Usage_PolicyDefinitionRequest_body('usage-policy-generic', policy_action="use") 

res_create_asset_usage_plicy = requests.post(url_edc_provider_policy_management, headers=header_control_plane, json=create_usage_policy_body)
res_create_asset_usage_plicy # post against: "{{EDCTX-10-1-URL}}/management/v2/policydefinitions"

In [11]:
# -----------------------------------------------------------------------------------------------------------------------

In [ ]:
# create contract:
contract_id = "demo_contract"
access_policy_id = create_access_policy_body['@id']
usage_policy_id  = create_usage_policy_body['@id']
asset_id         = create_asset_body['@id']


create_contract_body = create_generic_ContractDefinitionRequest_body(asset_id, contract_id, access_policy_id, usage_policy_id)

res_create_asset_contract = requests.post(url_edc_provider_contract_management, headers=header_control_plane, json=create_contract_body)
res_create_asset_contract # post against: "url": "{{EDCTX-10-1-URL}}/management/v2/contractdefinitions"

In [ ]:
# - which policies are registered with the provider -
res_policies = requests.post(url_edc_provider_contract_management + "/request", headers=header_control_plane)
print(res_policies)

print("{idx:8} {id:28} {asset_id:42} {accespolicy:28} {usagepolicy:28}".format(idx="Index", id="id", asset_id="Asset", accespolicy="Access Policy", usagepolicy="Usage Policy"))
print("-"*8 + " " + "-"*28 + " " + "-"*42 + " " + "-"*28 + " " + "-"*28)

for idx, retrived_contract in enumerate(res_policies.json()):
    # get asset
    try:    # please don't do this ...
        asset_ = retrived_contract['assetsSelector'][0]['operandRight']
    except:
        try:
            asset_ = retrived_contract['assetsSelector']['operandRight']
        except:
            asset_ = "None"
    print("{idx:8} {id:28} {asset_id:42} {accespolicy:28} {usagepolicy:28}".format(idx=str(idx)+ ":", id=retrived_contract["@id"], 
                                                asset_id=asset_,
                                                accespolicy=retrived_contract['accessPolicyId'], usagepolicy=retrived_contract['contractPolicyId']))